# Curve Fitting Tool (Univariate Functions)

<strong>This Jupyter Notebook serves as a tool for obtaining the optimum parameters of a defined function to fit some input data. This tool is valid only for univariate functions.</strong>

# 1. Install necessary libraries

In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} ipywidgets=7.5.1

In [ ]:
!conda install --yes --prefix {sys.prefix} numpy
!conda install --yes --prefix {sys.prefix} pandas
!conda install --yes --prefix {sys.prefix} matplotlib
!conda install --yes --prefix {sys.prefix} scipy
!conda install --yes --prefix {sys.prefix} xlrd
!conda install --yes --prefix {sys.prefix} IPython

In [ ]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
import ipywidgets as widgets
from IPython.display import display

# 2. Import the data file

<strong>Input the name and path of the file where your dataset is stored. The input file should be a .xlsx file with the data located in the first and second column [x, y].</strong>

In [ ]:
file_name = 'xy_dataset_SCC.xlsx'

**Input the name of the Excel sheet where the data is located. The code assumes the data headings are located in the first row.**

In [ ]:
sheet_name = 'Sheet1'

In [ ]:
df = pd.read_excel(str(file_name), sheet_name=sheet_name, header=0)

# 3. Explore and prepare the data

<strong>Visualize your data</strong>

In [ ]:
print(df)

<strong>Showing some data stats</strong>

In [ ]:
df.describe()

<strong>Converting dataframe columns to list type</strong>

In [ ]:
xdata = df[df.columns[0]].tolist()
ydata = df[df.columns[1]].tolist()

<strong>Converting lists to arrays</strong>

In [ ]:
xdata = np.array(xdata)
ydata = np.array(ydata)

# 4. Select the function to fit the data

<strong>This section allows you to choose the function that will be used to fit the data. If you want to change the default function just modify the python function statement below accordingly. Further predetermined functions will be added in the future.</strong>

<strong>Linear Function</strong>
$$f(x) = ax + b$$

In [ ]:
def lin_func(x, a, b):
    return a*x + b

<strong>Eight Degree Polynomial</strong>
$$f(x) = ax^8+bx^7+cx^6+dx^5+ex^4+fx^3+gx^2+hx^1+ix^0$$

In [ ]:
def poly8_func(x, a, b, c, d, e, f, g, h, i):
    return a*x**8 + b*x**7 + c*x**6 + d*x**5 + e*x**4 + f*x**3 + g*x**2 + h*x**1 + i*x**0

<strong>Seventh Degree Polynomial</strong>
$$f(x) = ax^7+bx^6+cx^5+dx^4+ex^3+fx^2+gx^1+hx^0$$

In [ ]:
def poly7_func(x, a, b, c, d, e, f, g, h):
    return a*x**7 + b*x**6 + c*x**5 + d*x**4 + e*x**3 + f*x**2 + g*x**1 + h*x**0

<strong>Sixth Degree Polynomial</strong>
$$f(x) = ax^6+bx^5+cx^4+dx^3+ex^2+fx^1+gx^0$$

In [ ]:
def poly6_func(x, a, b, c, d, e, f, g):
    return a*x**6 + b*x**5 + c*x**4 + d*x**3 + e*x**2 + f*x**1 + g*x**0

<strong>Fifth Degree Polynomial</strong>
$$f(x) = ax^5+bx^4+cx^3+dx^2+ex^1+fx^0$$

In [ ]:
def poly5_func(x, a, b, c, d, e, f):
    return a*x**5 + b*x**4 + c*x**3 + d*x**2 + e*x**1 + f*x**0

<strong>Fourth Degree Polynomial</strong>
$$f(x) = ax^4+bx^3+cx^2+dx^1+ex^0$$

In [ ]:
def poly4_func(x, a, b, c, d, e):
    return a*x**4 + b*x**3 + c*x**2 + d*x**1 + e*x**0

<strong>Third Degree Polynomial</strong>
$$f(x) = ax^3+bx^2+cx^1+dx^0$$

In [ ]:
def poly3_func(x, a, b, c, d):
    return a*x**3 + b*x**2 + c*x**1 + d*x**0

<strong>Second Degree Polynomial</strong>
$$f(x) = ax^2+bx^1+cx^0$$

In [ ]:
def poly2_func(x, a, b, c):
    return a*x**2 + b*x**1 + c*x**0

<strong>Logarithmic Function</strong>
$$f(x) = a + bln(x)$$

In [ ]:
def log_func(x, a, b):
    return a + b*np.log10(x)

<strong>Exponential Function</strong>
$$f(x) = ae^{bx}$$

In [ ]:
def exp_func(x, a, b):
    return a*np.exp(b*x)

<strong>Exponential Function for REE</strong>
$$f(t) = a(1-e^{-t/b})$$

In [ ]:
def expree_func(x, a, b):
    return a*(1-np.exp(-x/b))

<strong>Gaussian Distribution</strong>
$$f(x) = \frac{1}{\sigma\sqrt{2\pi}}e^{-\frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^{2}}$$

In [ ]:
def gaussian_func(x, a, b):
    return (1/(a*np.sqrt(2*np.pi)))*np.exp(-(x - b)/a)**2

<strong>Fourier Series</strong>
$$f(x) = \frac{a_0}{2} + \sum^{\infty}_{k=1}a_kcos\left(\frac{2\pi kx}{T}\right) + \sum^{\infty}_{k=1}b_ksin\left(\frac{2\pi kx}{T}\right)$$

In [ ]:
def func(x, w, a0, a1, a2, a3, b1, b2, b3):
    return a0 + a1*np.cos(w*x) + a2*np.cos(2*w*x) + a3*np.cos(3*w*x) + b1*np.sin(w*x) + b2*np.sin(2*w*x) + b3*np.sin(3*w*x)

In [ ]:
function = widgets.Dropdown(
    options=['Linear',
             'Polynomial',
             'Logarithmic',
             'Exponential',
             'REE Custom Function',
             'Gaussian'],
    value='Polynomial',
    description='Function:',
    disabled=False)
display(function)

In [ ]:
if function.value=='Polynomial':
    func=poly8_func
    init_vals = None
elif function.value=='Linear':
    func=lin_func
    init_vals = None
elif function.value=='Logarithmic':
    func=log_func
    init_vals = None
elif function.value=='Exponential':
    func=exp_func
    init_vals = None
elif function.value=='REE Custom Function':
    func=expree_func
    init_vals = None
elif function.value=='Gaussian':
    func=gaussian_func
    init_vals = None

<strong>Select the degree of the polynomial to use. If you didn't select a polynomial function to fit the data you can skip this step.</strong>

<strong>At this moment, only polynomials of degree 2 to 8 can be used.</strong>

In [ ]:
print(function.value)

if function.value=='Polynomial':
    
    deg = widgets.IntText(value=8,
                          description='Degree:',
                          disabled=False)
    display(deg) 

In [ ]:
if function.value=='Polynomial':
    if deg.value==8:
        func=poly8_func
        init_vals = None
    elif deg.value==7:
        func=poly7_func
        init_vals = None
    elif deg.value==6:
        func=poly6_func
        init_vals = None
    elif deg.value==5:
        func=poly5_func
        init_vals = None
    elif deg.value==4:
        func=poly4_func
        init_vals = None
    elif deg.value==3:
        func=poly3_func
        init_vals = None
    elif deg.value==2:
        func=poly2_func
        init_vals = None

In [ ]:
arg_list_comp = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k',
                 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
                 'w', 'x', 'y', 'z']

arg_list = arg_list_comp[0:deg.value+1]

In [ ]:
print(deg.value)
print(type(deg.value))

In [ ]:
print(func)

# 5. Method used to find the optimum coefficients

## Least Squares Minimization

<strong>For the algorithm to work properly initial values should be in the order of magnitude of the max 'x' and 'y' values respectively.</strong>

In [ ]:
#init_vals = [1, 1, 1, 1, 1, 1, 1, 1, 1]

<strong>
    
The default algorithm to perform the least-squares minimisation is the Levenberg-Marquardt algorithm, this algorithm doesn’t handle bounds and sparse Jacobians. Usually the most efficient method for small unconstrained problems.

For large sparsed bounded problems, the trf : Trust Region Reflective algorithm is more suitable and generally more robust method.

‘dogbox’ : dogleg algorithm with rectangular trust regions, typical use case is small problems with bounds. Not recommended for problems with rank-deficient Jacobian.
    
The method ‘lm’ won’t work when the number of observations is less than the number of variables, use ‘trf’ or ‘dogbox’ in this case.

To choose the method more suitable for your problem just add method='trf', 'dogbox' or 'lm' after the last argument in the below line. If no method is specified, the Levenberg-Marquardt is going to be used.
    
To select the method, just choose from the dropdown list below.

</strong>

In [ ]:
algorithm = widgets.Dropdown(
    options=['Levenberg-Marquardt',
             'Trust Region Reflective',
             'Dogleg'],
    value='Levenberg-Marquardt',
    description='Algorithm:',
    disabled=False)
display(algorithm)

In [ ]:
if algorithm.value=='Levenberg-Marquardt':
    method='lm'
elif algorithm.value=='Trust Region Reflective':
    method='trf'
elif algorithm.value=='Dogleg':
    method='dogbox'

In [ ]:
popt, pvar = opt.curve_fit(func, xdata, ydata,  p0=init_vals, method=method)

# 6. Results

In [ ]:
np.set_printoptions(formatter={'float': '{: 0.15f}'.format})
print ('Optimum Model Coefficients:')
print()
i = 0
for c in popt:
    if c > 0:
        print('%s =  %s' % (arg_list[i], c))
        i = i + 1
    else:
        print('%s = %s' % (arg_list[i], c))
        i = i + 1

In [ ]:
data_fitted = func(xdata, *popt)
xarray = np.linspace(0, max(xdata), 800)

<strong> RMS error of fit </strong>

In [ ]:
RMSE = np.sqrt(np.mean((data_fitted - ydata) ** 2))
print ('RMS Error: ', RMSE)

**Visualize the results**

In [ ]:
plt.figure(figsize=(9, 6), dpi=200)
plt.scatter(xdata, ydata, color='#e74c3c', label='Test Data')
plt.plot(xarray, func(xarray, *popt), color='#3498db', label='Fitted Data')
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
print('xdata'+11*' '+'ydata'+11*' '+'data_fitted'+10*' '+'RMSE')
for i in range(len(xdata)):
    print((('%5.4f'+10*' '+'%5.4f'+10*' '+'%5.4f'+15*' '+'%5.10f') % (xdata[i], ydata[i], data_fitted[i], np.sqrt(np.mean((data_fitted[i] - ydata[i]) ** 2)))))